In [ ]:
#----------------------------------------------------------------------
# post로 문서분류 하는 예시
#
# http://10.10.4.10:9001/docs/label
#
#curl -X 'POST' \
#  'http://10.10.4.10:9001/docs/label?infilepath=false' \
#  -H 'accept: application/json' \
#  -H 'Content-Type: application/json' \
#  -d '{
#  "Docs": {
#    "uids": [
#      "01","02"
#    ],
#    "titles": [
#      "대구환경청, 내년 3월까지 '\''겨울철 철새 먹이주기'\'' 활동", "국가정보자원관리원 대구센터 착공…전자정부 핵심기관"
#    ],
#    "documents": [
#      "대구지방환경청은 21일 '\''겨울철 철새 먹이주기 활동사업'\''에 나선다고 밝혔다.철새 도래지인 낙동강 해평습지와 금호강 안심습지 등에 서식하는 두루미류 등 겨울철새의 안전한 월동에 도움을 주기 위해서다.대구환경청은 내년 3월까지 구미 낙동강과 경산 금호강 일대에서 총 16회에 걸쳐 곡식과 고구마 등 총 2t을 공급한다.(이름) 대구환경청장은 '\''겨울철 야생동물이 건강하게 봄을 맞이할 수 있도록 보호활동을 더욱 적극적으로 추진하겠다'\''고 말했다.","각급 정부기관의 통신망을 관리할 국가정보자원관리원 대구센터가 31일 대구 동구 도학동에서 착공했다.4312억원을 투입해 2021년 8월 준공 예정인 대구센터는 86개 기관의 서비와 장비를 운용하게 된다.정부통합전산센터에서 명칭이 변경된 국가정보자원관리원은 현재 대전본원과 광주센터를 운영 중이다.대구센터는 급변하는 행정환경과 4차 산업혁명 시대에 맞춰 클라우드, 빅데이터 등 신기술이 접목된 지능형 전산센터로 구축된다.대구센터가 가동되면 매년 국비 1000억원 (이름) (이름) 사업 발주와 600여명의 정부 기관 인력이 상주해 4000억원의 생산유발효과, 2000억원의 부가가치유발효과가 나타날 것으로 대구시는 전망했다.특히 대구센터는 영남권 ict 융합 확산을 주도하고 우수 인재를 결집해 새 성장동력을 창출하는데 기여할 것으로 예상된다.(이름) 대구시 행정부시장은 '\''대구센터가 대구 전체 발전을 이끄는 동력이 되도록 진입도로 정비 등 기반시설 조성을 차질없이 추진하겠다'\''고 말했다."
#    ]
#  },
#  "Labels": {
#    "label_uids": [
#      "011", "012", "013", "014", "015", "016", "017", "018", "019"
#    ],
#    "label_text": [
#      "경제","사회일반","정치","환경", "정치,정당","사건,사고", "교육", "예술,문화", "스포츠,연애"
#    ]
#  }
#}''
#----------------------------------------------------------------------
import requests
import json

FASTAPI_URL = "http://10.10.4.10:9001/"

# 분류할 문서 내용(파일경로)
UID_LIST = ["1001"]
TITLE_LIST = ["s1"]

# 분류라벨
LABELS_UID_LIST = ["011","012","013","014", "015", "016", "017", "018", "019", "020"] #라벨 uid
LABELS_LIST = ["경제", "사회일반", "정치", "환경/자연", "정치/정당", "사건/사고", "교육", "예술/문화", "스포츠/연애", "관광/여행/숙박"] #라벨 name

def run_embedding_query_loop():
    while True:
        try:
            handle_query_embedding()
        except KeyboardInterrupt:
            return
        
def handle_query_embedding():
    
     #headers
    headers = {
        "Content-Type": "application/json"
    }

    #data
    #query = "my name is bongsoo. who are you?"
    #query = "안녕하세요.지금은 curl 테스트 입니다."
    document = input("분류할 문장 입력: ")
    document = [document]
    print()
    
    url = FASTAPI_URL + "docs/label?infilepath=false"
    print(f'url:{url}\n')

    # body 리스트
    temp = {
        "Docs": {
            "uids":UID_LIST,
            "titles":TITLE_LIST,
            "documents":document
        },
        "Labels": {
            "label_uids":LABELS_UID_LIST,
            "label_text":LABELS_LIST
        }
    }

    # dict 을 json으로 변환
    data = json.dumps(temp)
    
    print('data')
    print(f'{data}\n')
    
    # header와 data 를 같이 전송
    try:
        response = requests.post(url, headers=headers, data=data)
    except requests.exceptions.Timeout as e:
        print("Timeout Error:", e)
        return
    except requests.exceptions.ConnectionError as e:
        print("Connection Error:", e)
        return
    except requests.exceptions.HTTPError as e:
        print("Http Error:", e)
        return
    except requests.exceptions.RequestException as e:#Any Error except
        print("AnyException Error:", e)
        return
    
    # 응답결과값 출력
    response_dict = json.loads(response.text)
    try:
        labels = response_dict["labels"]
    except Exception as e:
        print("response_dict Error:", e)
        print(response_dict)
        return
    
    print(f'\n*응답결과\n{labels}\n') 
    
    for label in labels:
        label_uid = label['uid']
        #print(label_uid)
        print(f'*분류라벨\n{LABELS_LIST[LABELS_UID_LIST.index(label_uid)]}({label["score"]})\n')
    
run_embedding_query_loop()